<a href="https://colab.research.google.com/github/lisaong/diec/blob/master/day3/swarm/pygmo_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Job Shop Problem

(Credits: https://developers.google.com/optimization/scheduling/job_shop)

One common scheduling problem is the job shop, in which multiple jobs are processed on several machines. Each job consists of a sequence of tasks, which must be performed in a given order, and each task must be processed on a specific machine. For example, the job could be the manufacture of a single consumer item, such as an automobile. The problem is to schedule the tasks on the machines so as to minimize the length of the schedule—the time it takes for all the jobs to be completed.

There are several constraints for the job shop problem:

* No task for a job can be started until the previous task for that job is completed.
* A machine can only work on one task at a time.
* A task, once started, must run to completion.

## How this applies to Edge Computing
Scalable Edge Computing architectures involves multiple compute nodes, where each node may specialise in different functions. For example, an airport security system where video cameras are used to track passenger movement can be envisioned as a pipeline of Edge nodes:
- Nodes attached to the video camera doing frame extraction and buffering
- Nodes receiving the buffered video frames to perform feature extraction
- Nodes running ML algorithms on the extracted video features to get the predicted passenger paths

Note that the term "nodes" here may refer to physical nodes or logical (functional) nodes within an Edge compute system. The idea is still the same - a pipeline is split into stages (tasks assigned to nodes) so that we can parallelise the pipeline (run multiple jobs at the same time).

The Job Shop problem is about how to schedule tasks in parallel to minimise overall latency.

## Example Problem

Below is a simple example of a job shop problem, in which each task is labeled by a pair of numbers (m, p) where m is the number of the machine the task must be processed on and p is the processing time of the task — the amount of time it requires. (The numbering of jobs and machines starts at 0.)

* job 0 = [(0, 3), (1, 2), (2, 2)]
* job 1 = [(0, 2), (2, 1), (1, 4)]
* job 2 = [(1, 4), (2, 3)]

In the example, job 0 has three tasks. The first, (0, 3), must be processed on machine 0 in 3 units of time. The second, (1, 2), must be processed on machine 1 in 2 units of time, and so on. Altogether, there are eight tasks.


### A solution for the problem
A solution to the job shop problem is an assignment of a start time for each task, which meets the constraints given above. The diagram below shows one possible solution for the problem:

![schedule](https://developers.google.com/optimization/images/scheduling/schedule1.png)

You can check that the tasks for each job are scheduled at non-overlapping time intervals, in the order given by the problem.

The length of this solution is 12, which is the first time when all three jobs are complete.

(credits: https://developers.google.com/optimization/scheduling/job_shop)

### Variables and constraints for the problem

This section describes how to set up the variables and constraints for the problem. First, let task(i, j) denote the jth task in the sequence for job i. For example, task(0, 2) denotes the second task for job 0, which corresponds to the pair (1, 2) in the problem description.

Next, define $t_{i, j}$ to be the start time for task(i, j). The $t_{i, j}$ are the variables in the job shop problem. Finding a solution involves determining values for these variables that meet the requirement of the problem.

There are two types of constraints for the job shop problem:

* Precedence constraints — These arise from the condition that for any two consecutive tasks in the same job, the first must be completed before the second can be started. For example, task(0, 2) and task(0, 3) are consecutive tasks for job 0. Since the processing time for task(0, 2) is 2, the start time for task(0, 3) must be at least 2 units of time after the start time for task 2. (Perhaps task 2 is painting a door, and it takes two hours for the paint to dry.) As a result, you get the following constraint:

    $t_{0, 2}   + 2  ≤  t_{0, 3}$

* No overlap constraints — These arise from the restriction that a machine can't work on two tasks at the same time. For example, task(0, 2) and task(2, 1) are both processed on machine 1. Since their processing times are 2 and 4, respectively, one of the following constraints must hold:
    $t_{0, 2}   + 2  ≤  t_{2, 1}$ (if task(0, 2) is scheduled before task(2, 1))

    or

    $t_{2, 1}   + 4  ≤  t_{0, 2}$ (if task(2, 1) is scheduled before task(0, 2)).

### Objective for the problem
The objective of the job shop problem is to minimize the makespan: the length of time from the earliest start time of the jobs to the latest end time

### PyGMO

The example from Google uses Google's or-tools CP-SAT solver  (https://developers.google.com/optimization/cp/cp_solver) to solve the scheduling problem.

We will see how biologically inspired Global Optimisation Algorithms can solve this problem, using the PyGMO library. PyGMO is a Python wrapper for the PaGMO (Parallel Global Multi-objective Optimizer) C++ solver. Source: https://github.com/esa/pygmo2

The Jobshop scheduling problem is known as a "constrainted single-objective optimisation problem." It means that:
* There is 1 objective (to minimise, or to maximise)
* There are constraints that limit the solution space

Not all algorithms perform constrained optimisation. Most algorithms in PyGMO support unconstrained optimisation.

Libraries with Similar Approaches for solving Scheduling Problems:
* https://pypi.org/project/pyschedule/0.2.6/
* https://github.com/mcfadd/Job_Shop_Schedule_Problem
* https://github.com/brandhaug/jssp-swarm (in Scala)


In [0]:
!pip install pygmo

     |████████████████████████████████| 10.7MB 460kB/s 


### A simple user-defined problem

As an introduction to how to use PyGMO, we'll first cover a simple problem of finding the values of $(x_1, x_2)$ that will minimise $y = {x_1}^2 + {x_2}^2$.



In [0]:
import pygmo as pg

# https://esa.github.io/pygmo2/tutorials/coding_udp_simple.html
class example_function:
  def __init__(self, dim):
    self.dim = dim

  def fitness(self, x):
    """The objective function to minimise
    """
    return [sum(x*x)]
  
  def get_bounds(self):
    """The bounds of the vector (the answer)
    """
    return ([-1] * self.dim,[1] * self.dim)

  def get_extra_info(self):
    """Describes extra information of the problem
    """
    return f'\tDimensions: {self.dim}'
  
prob = pg.problem(example_function(dim=2))

In [0]:
print(prob)

Problem name: <class '__main__.example_function'>
	Global dimension:			2
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [-1, -1]
	Upper bounds: [1, 1]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none

Extra info:
	Dimensions: 2


In [0]:
algo = pg.algorithm(pg.bee_colony(gen = 20, limit = 20))
pop = pg.population(prob,10)
pop = algo.evolve(pop)
print(f'solution: {pop.champion_x}, fitness value: {pop.champion_f}')

solution: [-0.00020625  0.00028443], fitness value: [1.23436773e-07]


### Jobshop Problem Definition
For the jobshop scheduling problem, we have 8 tasks.

As the jobs_data is static, we can express the solution as a list of start times, one start time per job.

Dimensions of our problem: 8

Constraints: 2
*   Precedence
*   No-overlap

In [0]:
# Each job is a list of multiple tasks: (machine_id, processing_time)
jobs_data = [
  [(0, 3), (1, 2), (2, 2)],  # Job0
  [(0, 2), (2, 1), (1, 4)],  # Job1
  [(1, 4), (2, 3)]  # Job2
]

machines_count = 1 + max(task[0] for job in jobs_data for task in job)
all_machines = range(machines_count)

In [0]:
# Flatten the jobs data by index by encoding the job_id into the tuple

# the *task syntax will unpack a tuple
jobs_list = [(i, *task) for i in range(len(jobs_data)) for task in jobs_data[i]]

jobs_list # (job_id, machine_id, processing_time)

[(0, 0, 3),
 (0, 1, 2),
 (0, 2, 2),
 (1, 0, 2),
 (1, 2, 1),
 (1, 1, 4),
 (2, 1, 4),
 (2, 2, 3)]

In [0]:
# now we can define our problem
import numpy as np

# constants for accessing the tuples
JOB = 0
MACHINE = 1
DURATION = 2

class jobshop_function:
  def __init__(self, jobs, max_schedule_time=20):
    """ jobs: list of jobs (job_id, machine_id, duration)
        max_schedule_time: maximum time allowed for the schedule
    """
    self.jobs = jobs_list
    self.dim = len(jobs_list)
    self.t_max = max_schedule_time
    self.t_min = 0

    self.job_start_indices = []
    for i in range(1, self.dim):
      if self.jobs[i-1][JOB] != self.jobs[i][JOB]:
        self.job_start_indices.append(i)    
    self.machines_count = 1 + max(job[MACHINE] for job in jobs)

  def fitness(self, x):
    """The fitness: the objective to minimize and any constraints
    x: contains the start times
    """
    # Compute the objective:
    # the length of time from the earliest start time of the jobs to the
    # latest end time
    start_times = x # code readability
    end_times = [job[DURATION]+s for job, s in zip(self.jobs, start_times)]
    objective = max(end_times) - min(start_times)
    assert(objective > 0)

    # Compute the constraints:
    # 1) Precedence: for any two consecutive tasks in the same job, the first must 
    # be completed before the second can be started
    #
    # Say we have 3 tasks with their start and end times:
    #   (s1, e1), (s2, e2), (s3, e3)
    # Check that:
    #   s1 >= 0, s2 >= e1, s3 >= e2
    # If any of these are FALSE, the constraint is not met

    # Compute the prev task's end times, initializing end times to
    # -1 for the first task of each job
    # (-1 is used instead of 0 to allow for start times at 0)
    prev_end_times = np.array([-1] + end_times[:-1])
    prev_end_times[self.job_start_indices] = -1

    # Count how many times the constraint is not met
    # (i.e. when prev task's end time exceeds next task's start time
    violate_precedence = float(sum((prev_end_times - start_times) > 0))
    
    # 2) No overlap: a machine can't work on two tasks at the same time.
    # Say we have 2 tasks on the same machine with their start and end times,
    # where the start times are sorted
    #   (s1, e1), (s2, e2)
    # Check that e1 <= s2. We can do this by sorting first by start times,
    # then flattening the tuples out to make sure that the values are sorted
    #
    # Extract tasks assigned to each machine, their start and end times
    tasks_by_machines = {machine:[] for machine in range(self.machines_count)}
    for i in range(self.dim):
      tasks_by_machines[self.jobs[i][MACHINE]].append(
          (start_times[i], end_times[i]))

    # sort by start time, then compute whether overlap happens
    # none of the numbers should overlap
    overlap = 0.
    for machine in tasks_by_machines.keys():
      tasks_by_machines[machine].sort(key=lambda t:t[0]) # inplace sort
      flattened = np.array([t for ts in tasks_by_machines[machine] for t in ts])
      # overlap means previous value exceeds the next value
      overlap += float(sum(flattened[:-1] > flattened[1:]))

    # subtract the earliest start time to get the makespan
    return [objective, violate_precedence, overlap]

  def get_bounds(self):
    """The bounds of the vector
    """
    return ([self.t_min] * self.dim,[self.t_max] * self.dim)

  def get_nix(self):
    """Returns the number of integer dimensions"""
    return self.dim

  def get_nec(self):
    """The number of equality constraints:
    1 no-overlap constraint
    1 precedence constraint
    """
    return 2

  def get_extra_info(self):
    return f'\tDimensions: {self.dim} \
      \n\tJobs: {self.jobs} \
      \n\tSchedule Limit: {self.t_max}'
  
jobshop_prob = pg.problem(jobshop_function(jobs=jobs_list))

In [0]:
print(jobshop_prob)

Problem name: <class '__main__.jobshop_function'>
	Global dimension:			8
	Integer dimension:			8
	Fitness dimension:			3
	Number of objectives:			1
	Equality constraints dimension:		2
	Inequality constraints dimension:	0
	Tolerances on constraints: [0, 0]
	Lower bounds: [0, 0, 0, 0, 0, ... ]
	Upper bounds: [20, 20, 20, 20, 20, ... ]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none

Extra info:
	Dimensions: 8       
	Jobs: [(0, 0, 3), (0, 1, 2), (0, 2, 2), (1, 0, 2), (1, 2, 1), (1, 1, 4), (2, 1, 4), (2, 2, 3)]       
	Schedule Limit: 20


In [0]:
# Unit testing
jobshop_prob.fitness([1, 2, 3, 4, 5, 6, 7, 8])

array([10.,  4.,  1.])

## Run the Optimiser

List of algorithms are here:
https://esa.github.io/pygmo2/overview.html#list-of-algorithms

GACO runs the Extended Ant Colony Optimization algorithm, which supports constrained optimisation:
https://esa.github.io/pygmo2/algorithms.html#pygmo.gaco


In [0]:
# Optimise
population_size = 1000
iterations = 100

prob = pg.problem(jobshop_function(jobs=jobs_list, max_schedule_time=12))
pop = pg.population(prob, size=population_size, seed=123)
algo = pg.algorithm(pg.gaco(gen=iterations, ker=population_size, seed=123))
algo.set_verbosity(1000)
pop = algo.evolve(pop)

print(f'solution: {pop.champion_x}, fitness value: {pop.champion_f}')

solution: [ 3.  6. 11.  1.  6.  9.  2.  7.], fitness value: [12.  0.  0.]


Original Problem:
```
(machine_id, duration)
jobs_data = [
  [(0, 3), (1, 2), (2, 2)],  # Job0
  [(0, 2), (2, 1), (1, 4)],  # Job1
  [(1, 4), (2, 3)]  # Job2
]
```

Schedule:
```
start_times = [
  [3, 6, 11], # Job0
  [1, 6, 9], # Job1
  [2, 7], # Job2
]
```

Machine 0: 
* Job1 Task 0 at t=1-2
* Job0 Task 0 at t=3-6

Machine 1:
* Job2 Task 0 at t=2-6
* Job0 Task 1 at t=6-8
* Job1 Task 2 at t=9-13

Machine 2:
* Job1 Task 1 at t=6-7
* Job2 Task 1 at t=7-10
* Job0 Task 2 at t=11-13

The overall scheduled time is 12, which is similar to the example by Google's OR tools. The constraint logic can be fine-tuned further as it is authored by hand.